In [1]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
from shapely.geometry import Point
import unicodedata

Steps:
1. Download data from sales force. Make sure to choose details only  and .xls format. 
2. Delete the "Current crossing method" column that contains the embeding codes <img...
3. Open in excel and save as a .csv file
4. Change the name of the input file to the name and path of the report
5. Change the name and path of the output file

## Data Clean up
This is code that cleans up data from salesforce and creates geojson for map

In [19]:
# Change Variables below
input_file = './home/data/report1679360399851.csv'
output_file = 'home/data/civ-assessments-v7.geojson'

In [45]:
gdf = gpd.read_file(input_file, encoding='utf-8-sig')
gdf['latitude'] = gdf['Bridge Opportunity: GPS (Latitude)']
gdf['longitude'] = gdf['Bridge Opportunity: GPS (Longitude)']
gdf['latitude'] = gdf['latitude'].astype(float)
gdf['longitude'] = gdf['longitude'].astype(float)
# set geometry to latitude and longitude columns
gdf['geometry'] = gdf.apply(lambda x: Point((float(x.longitude), float(x.latitude))), axis=1)
gdf.columns = gdf.columns.str.strip()
gdf = gdf.drop(columns=['Current crossing method', "Site Validation Photo URL", 'High water line URL', 'Current crossing method URL', 'Current crossing method', "Site Validation Photo", "Rejection Image 1", "Rejection Image 2", "Current crossing method URL", "High water line URL", "Downstream URL", "Downstream", "Upstream URL", "Upstream", "Left top of bank (toward) URL", "Left top of bank (toward)", "Left top of bank (away) URL", "Left top of bank (away)", "Right top of bank (toward) URL", "Right top of bank (toward)", "Right top of bank (away) URL", "Right top of bank (away)", "Form: Created By"])
# remove all rows that have "Duplicate" in the "Bridge Opportunity: Sub-Stage" column
print ("Dataframe is " + str(len(gdf)) + ", rows in length")
gdf = gdf[gdf['Bridge Opportunity: Sub-Stage'] != 'Duplicate']
print ("Dataframe is " + str(len(gdf)) + " rows in length")
gdf.columns = gdf.columns.str.replace('Bridge Opportunity:', '')
gdf.columns = gdf.columns.str.strip()



gdf.to_file(output_file, driver='GeoJSON', encoding='utf-8-sig')
gdf.to_csv('home/data/civ-assessments-v7.csv', encoding='utf-8-sig')

Dataframe is 676, rows in length
Dataframe is 624 rows in length


In [35]:
# return row 87 of df
gdf.iloc[218]

Opportunity Name                                            CÃ´te d'Ivoire - KambirÃ© - 1043454
Opportunity Unique Identifier                                                           1043454
Bridge Name                                                                            KambirÃ©
Country                                                                          CÃ´te d'Ivoire
Level 3 Government                                                                   DJOUROUTOU
Level 4 Government                                                                      Mahino1
Stage                                                                                Identified
Sub-Stage                                                                                      
Individuals Directly Served                                                                    
River Name                                                                                 Neka
Site Validation                         

## This code is used to create unique clustering

In [26]:
# convert "River crossing deaths in last 3 years" to int
filter_gdf = gdf
filter_gdf['River crossing deaths in last 3 years'] = filter_gdf['River crossing deaths in last 3 years'].astype(int)
# create a dataframe with rows where "River crossing deaths in last 3 years" is greater than 0
mgdf = filter_gdf[filter_gdf['River crossing deaths in last 3 years'] > 0]
# create a dataframe with rows where "Flag for Rejection" No
ngdf = filter_gdf[filter_gdf['Flag for Rejection'] == 'No']
# create a dataframe from mgdf where "Flag for Rejection" is No
mngdf = mgdf[mgdf['Flag for Rejection'] == 'No']
# write mgdf, ngdf, and mn gdf to geojson files
mgdf.to_file(output_file[:-8]+'-mgdf.geojson', driver='GeoJSON')
ngdf.to_file(output_file[:-8]+'-ngdf.geojson', driver='GeoJSON')
mngdf.to_file(output_file[:-8]+'-mngdf.geojson', driver='GeoJSON')

## Exports all data to shape files

In [ ]:
import os
for files in os.listdir('./home/data/datawork'):
    # type is the last 8 characters of files
    filetype = files[-8:]
    filename = files[:-8]
    print (files)
    # read file as geodataframe
    if filetype == '.geojson':
        tempgdf = gpd.read_file('./home/data/datawork/' + files, encoding='iso-8859-1')        
        # write to csv
        tempgdf.to_csv('./home/data/outputs/' + filename + '.csv')
        # write to shp file
        tempgdf.to_file('./home/data/outputs/' + filename + '.shp', driver='ESRI Shapefile')